<a href="https://colab.research.google.com/github/psrathi24/Clinical_Dashboard_Project/blob/main/Clinical_Data_Tableau_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Dataset](https://lab.data.ca.gov/dataset/ambulatory-surgery-characteristics-by-patient-county-of-residence)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
surgery_disposition = pd.read_csv("ambulatory-surgery-characteristics-disposition-by-patient-county-of-residence.csv")
surgery_expected_payer = pd.read_csv("ambulatory-surgery-characteristics-expected-payer-by-patient-county-of-residence.csv")
surgery_race = pd.read_csv("ambulatory-surgery-characteristics-race-group-by-patient-county-of-residence.csv")
surgery_sex = pd.read_csv("ambulatory-surgery-characteristics-sex-by-patient-county-of-residence.csv")
pd.set_option('display.width', None)

## Data Cleaning:

In [ ]:
surgery_disposition.info()
surgery_disposition.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683 entries, 0 to 682
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patcnty1    683 non-null    object
 1   serv_y      683 non-null    int64 
 2   disp1       683 non-null    object
 3   Encounters  683 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 21.5+ KB


,patcnty1,serv_y,disp1,Encounters
0,Alameda,2023,Acute Care,322
1,Alameda,2023,Childrens_or_Cancer,3
2,Alameda,2023,Critical Access Hospital,1
3,Alameda,2023,Died,9
4,Alameda,2023,Home Health Service,2425


In [ ]:
# Update column names:
surgery_disposition.rename(columns={'serv_y': 'Service Year'}, inplace=True)
surgery_disposition.rename(columns={'patcnty1': 'County'}, inplace=True)
surgery_disposition.rename(columns={'disp1': 'Disposition'}, inplace=True)

# Update categorical column values:
surgery_disposition.replace({'Disposition': {'Hospice_Care': 'Hospice Care'}}, inplace=True)
surgery_disposition.replace({'Disposition': {'Childrens_or_Cancer': "Children's or Cancer-specific Care"}}, inplace=True)
surgery_disposition.replace({'Disposition': {'Prison/Jail': 'Correctional Facility'}}, inplace=True)
surgery_disposition.replace({'Disposition': {'Psychiatric_Care': 'Psychiatric Care'}}, inplace=True)
surgery_disposition.replace({'Disposition': {'Disaster_Care_Site': 'Disaster Care Site'}}, inplace=True)
surgery_disposition.replace({'Disposition': {'Not_Defined_Elsewhere': 'Other'}}, inplace=True) # Combined Not_Defined_Elsewhere & Other

In [ ]:
surgery_expected_payer.info()
surgery_expected_payer.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1014 entries, 0 to 1013
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   patcnty1    1014 non-null   object
 1   serv_y      1014 non-null   int64 
 2   payer1      1014 non-null   object
 3   Encounters  1014 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 31.8+ KB


,patcnty1,serv_y,payer1,Encounters
0,Alameda,2023,Blue Cross/Blue Shield,411
1,Alameda,2023,CHAMPUS (TRICARE),163
2,Alameda,2023,Commercial Insurance Company,7029
3,Alameda,2023,Exclusive Provider Organization (EPO),2058
4,Alameda,2023,Health Maintenance Organization,28012


In [ ]:
# Update column names:
surgery_expected_payer.rename(columns={'serv_y': 'Service Year'}, inplace=True)
surgery_expected_payer.rename(columns={'patcnty1': 'County'}, inplace=True)
surgery_expected_payer.rename(columns={'payer1': 'Payer'}, inplace=True)

# Update column values
def categorize_payer(payer):
    if payer in ['Blue Cross/Blue Shield', 'Preferred Provider Organization (PPO)', 'Commercial Insurance Company',
                  'Health Maintenance Organization (HMO) Medicare Risk', 'Health Maintenance Organization',
                  'Exclusive Provider Organization (EPO)', 'Point of Service (POS)']:
        return 'Private Insurance'
    elif payer in ['Medi-Cal', 'Medicare Part B', 'CHAMPUS (TRICARE)', 'Veterans Affairs Plan',
                    'Other Non-federal Programs', 'Other Federal Program', 'Title V', 'Medicare Part A']:
        return 'Government Programs'
    elif payer in ['Workers\' Compensation Health Claim', 'Automobile Medical']:
        return 'Workers\' Compensation'
    elif payer == 'Self Pay':
        return 'Self-Pay'
    else:
        return 'Other'

surgery_expected_payer['Payer Type'] = surgery_expected_payer['Payer'].apply(categorize_payer)

In [ ]:
surgery_race.info()
surgery_race.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552 entries, 0 to 551
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   patcnty1        552 non-null    object 
 1   serv_y          552 non-null    int64  
 2   race_grp1       552 non-null    object 
 3   Encounters      524 non-null    float64
 4   AnnotationCode  28 non-null     float64
 5   AnnotationDesc  28 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 26.0+ KB


,patcnty1,serv_y,race_grp1,Encounters,AnnotationCode,AnnotationDesc
0,Alameda,2023,American Indian/Alaska Native,222.0,NaN,NaN
1,Alameda,2023,Asian,24097.0,NaN,NaN
2,Alameda,2023,Black,10380.0,NaN,NaN
3,Alameda,2023,Hispanic,18235.0,NaN,NaN
4,Alameda,2023,Invalid/Blank,2.0,NaN,NaN


In [ ]:
# Update column names:
surgery_race.rename(columns={'serv_y': 'Service Year'}, inplace=True)
surgery_race.rename(columns={'patcnty1': 'County'}, inplace=True)
surgery_race.rename(columns={'race_grp1': 'Race'}, inplace=True)

# Update column values:
surgery_race["Encounters"] = surgery_race["Encounters"].replace([np.inf, -np.inf], np.nan).fillna(0).astype(int)
  # Update "Encounters" null values with 0 and update column type to int
surgery_race.replace({'Race': {'Invalid/Blank': 'Unknown'}}, inplace=True)

# Drop useless columns:
surgery_race.drop(columns = ['AnnotationCode', 'AnnotationDesc'], inplace = True)

In [ ]:
surgery_sex.info()
surgery_sex.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   patcnty1        161 non-null    object 
 1   serv_y          161 non-null    int64  
 2   gender          161 non-null    object 
 3   Encounters      161 non-null    int64  
 4   AnnotationCode  0 non-null      float64
 5   AnnotationDesc  0 non-null      float64
dtypes: float64(2), int64(2), object(2)
memory usage: 7.7+ KB


,patcnty1,serv_y,gender,Encounters,AnnotationCode,AnnotationDesc
0,Alameda,2023,Female,54339,NaN,NaN
1,Alameda,2023,Male,45453,NaN,NaN
2,Alameda,2023,Unknown/Invalid/Blank,21,NaN,NaN
3,Alpine,2023,Female,15,NaN,NaN
4,Alpine,2023,Male,25,NaN,NaN


In [ ]:
# Update column names:
surgery_sex.rename(columns={'serv_y': 'Service Year'}, inplace=True)
surgery_sex.rename(columns={'patcnty1': 'County'}, inplace=True)
surgery_sex.rename(columns={'gender': 'Gender'}, inplace=True)

# Update column values:
surgery_sex.replace({'Gender': {'Unknown/Invalid/Blank': 'Unknown'}}, inplace=True)

# Drop useless columns:
surgery_sex.drop(columns = ['AnnotationCode', 'AnnotationDesc'], inplace = True)

## Export Updated Dataframes

In [ ]:
surgery_disposition.to_excel('surgery_disposition.xlsx', index=False)
surgery_expected_payer.to_excel('surgery_expected_payer.xlsx', index=False)
surgery_race.to_excel('surgery_race.xlsx', index=False)
surgery_sex.to_excel('surgery_sex.xlsx', index=False)
public_vs_private.to_excel('public_vs_private.xlsx', index=True)

## Exploratory Data Analysis (EDA):

Patient Distributions Per County:

In [ ]:
# By Disposition:
county_distribution_1 = surgery_disposition.groupby('County')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
print(county_distribution_1)

print()

# By Expected Payer:
county_distribution_2 = surgery_expected_payer.groupby('County')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
print(county_distribution_2)

print()

# By Race:
county_distribution_3 = surgery_race.groupby('County')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
print(county_distribution_3)

print()

# By Sex:
county_distribution_4 = surgery_sex.groupby('County')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
print(county_distribution_4)

                   County  Encounters
19            Los Angeles      498219
38              San Diego      219211
30                 Orange      147561
44            Santa Clara      132740
34              Riverside      128278
37         San Bernardino       99867
0                 Alameda       99813
9                  Fresno       78102
35             Sacramento       75369
6            Contra Costa       58441
39          San Francisco       58363
42              San Mateo       57904
40            San Joaquin       51725
57                Ventura       49520
51             Stanislaus       48457
15                   Kern       48374
50                 Sonoma       47395
55                 Tulare       31732
49                 Solano       31631
32                 Placer       27747
45             Santa Cruz       25325
43          Santa Barbara       24240
3                   Butte       22695
31  Out of State/Homeless       22629
27               Monterey       21066
24          

Compared all 4 dataframes to ensure data is the same, which didn't turn out to be the case. This is strange because each table should contain the same number of cases per county, since data is only separated by a different characteristic for each table.

In [ ]:
print(county_distribution_1.equals(county_distribution_2))
print(county_distribution_1.equals(county_distribution_3))
print(county_distribution_1.equals(county_distribution_4))

True
False
True


In [ ]:
# Mismatched rows:

# Merge the two DataFrames on 'County' to compare Encounters values
merged_df = pd.merge(county_distribution_1, county_distribution_3, on='County', suffixes=('_df1', '_df2'))

# Filter rows where 'Encounters' values are different:
mismatched_rows = merged_df[merged_df['Encounters_df1'] != merged_df['Encounters_df2']]
mismatched_rows

,County,Encounters_df1,Encounters_df2
48,Glenn,2803,2794
49,Plumas,1917,1906
50,Inyo,1857,1844
51,Del Norte,1754,1732
52,Colusa,1747,1742
53,Mariposa,1500,1485
54,Mono,1236,1221
56,Trinity,955,945
57,Modoc,450,443
58,Sierra,264,260


Without access to patient records for those counties, I can't update the values. So I'll go with the values from county_distribution_1 as my official patient distribution per county.

In [ ]:
# Counties w/ Highest & Lowest Patient Volumes
top_counties = county_distribution_1.head()
bottom_counties = county_distribution_1.tail()
top_counties, bottom_counties

(         County  Encounters
 19  Los Angeles      498219
 38    San Diego      219211
 30       Orange      147561
 44  Santa Clara      132740
 34    Riverside      128278,
      County  Encounters
 18   Lassen        1068
 54  Trinity         955
 25    Modoc         450
 47   Sierra         264
 1    Alpine          40)

Demographics Breakdown By Sex & Race:

In [ ]:
race_distribution = surgery_race.groupby('Race')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
race_distribution

,Race,Encounters
8,White,1141290
3,Hispanic,688591
1,Asian,242345
2,Black,117012
6,Other,101735
7,Unknown,33526
4,Multi-racial,25225
5,Native Hawaiian/Other Pacific Islander,12186
0,American Indian/Alaska Native,7692


In [ ]:
sex_distribution = surgery_sex.groupby('Gender')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
sex_distribution

,Gender,Encounters
0,Female,1296449
1,Male,1072930
2,Unknown,342


Disposition Distribution:

In [ ]:
disposition_distribution = surgery_disposition.groupby('Disposition')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
disposition_distribution

,Disposition,Encounters
14,Routine (Home),2298667
6,Home Health Service,47895
15,Skilled Nursing/Intermediate Care (SN/IC),6451
2,Correctional Facility,5611
0,Acute Care,4842
9,Left Against Medical Advice,1472
13,Residential Care facility,1287
7,Hospice Care,977
10,Other,879
12,Rehab,709


In [ ]:
# Top Disposition By County:
top_disposition_by_county = surgery_disposition.groupby(['County', 'Disposition'])['Encounters'].sum().reset_index()
top_disposition_by_county = top_disposition_by_county.loc[top_disposition_by_county.groupby('County')['Encounters'].idxmax()]
top_disposition_by_county

,County,Disposition,Encounters
13,Alameda,Routine (Home),96588
15,Alpine,Routine (Home),40
23,Amador,Routine (Home),3451
32,Butte,Routine (Home),22134
39,Calaveras,Routine (Home),5594
44,Colusa,Routine (Home),1724
58,Contra Costa,Routine (Home),55650
65,Del Norte,Routine (Home),1685
75,El Dorado,Routine (Home),16404
90,Fresno,Routine (Home),76057


Payer Distribution By Payer & Payer Type:

In [ ]:
# By Payer:
payer_distribution_1 = surgery_expected_payer.groupby('Payer')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
payer_distribution_1

,Payer,Encounters
10,Medicare Part B,493193
8,Medi-Cal,444504
5,Health Maintenance Organization,425971
6,Health Maintenance Organization (HMO) Medicare...,381437
15,Preferred Provider Organization (PPO),343220
3,Commercial Insurance Company,77136
1,Blue Cross/Blue Shield,74897
16,Self Pay,22307
14,Point of Service (POS),19909
4,Exclusive Provider Organization (EPO),19256


In [ ]:
# By Payer Type:
payer_distribution_2 = surgery_expected_payer.groupby('Payer Type')['Encounters'].sum().reset_index().sort_values(by='Encounters', ascending=False)
payer_distribution_2

,Payer Type,Encounters
2,Private Insurance,1341826
0,Government Programs,984028
3,Self-Pay,22307
4,Workers' Compensation,18486
1,Other,3074


In [ ]:
# Calculate the proportion of each payer type per county:

# Total encounters per county:
total_encounters_per_county = surgery_expected_payer.groupby('County')['Encounters'].sum()
total_encounters_per_county

# Group by County and Payer Type to sum encounters
payer_type_by_county = surgery_expected_payer.groupby(['County', 'Payer Type'])['Encounters'].sum().reset_index()

# Merge total encounters per county
payer_type_by_county['Total County Encounters'] = payer_type_by_county['County'].map(total_encounters_per_county)

# Calculate the proportion of each payer type within each county
payer_type_by_county['Proportion'] = payer_type_by_county['Encounters'] / payer_type_by_county['Total County Encounters']

payer_type_by_county.head(20)

,County,Payer Type,Encounters,Total County Encounters,Proportion
0,Alameda,Government Programs,30865,99813,0.309228
1,Alameda,Other,81,99813,0.000812
2,Alameda,Private Insurance,67616,99813,0.677427
3,Alameda,Self-Pay,791,99813,0.007925
4,Alameda,Workers' Compensation,460,99813,0.004609
5,Alpine,Government Programs,25,40,0.625000
6,Alpine,Private Insurance,14,40,0.350000
7,Alpine,Workers' Compensation,1,40,0.025000
8,Amador,Government Programs,2263,3803,0.595057
9,Amador,Other,1,3803,0.000263


In [ ]:
# Calculate Public v.s. Private Insurance Proportions:

# Define public and private insurance categories
public_insurance_types = ['Government Programs', "Other"]
private_insurance_types = ['Private Insurance', "Workers' Compensation", "Self-Pay"]

# Aggregate proportions for public and private insurance
public_insurance = payer_type_by_county[payer_type_by_county['Payer Type'].isin(public_insurance_types)]
private_insurance = payer_type_by_county[payer_type_by_county['Payer Type'].isin(private_insurance_types)]

# Sum public and private proportions per county
public_vs_private = pd.merge(public_insurance.groupby('County')['Proportion'].sum(),
                             private_insurance.groupby('County')['Proportion'].sum(),
                             on = 'County', how = 'outer', suffixes = ('_Public', '_Private')).fillna(0)

# Compute the ratio of public to private insurance users per county
public_vs_private['Public_to_Private_Ratio'] = public_vs_private['Proportion_Public'] / public_vs_private['Proportion_Private']

public_vs_private.head(20)

,Proportion_Public,Proportion_Private,Public_to_Private_Ratio
County,,,
Alameda,0.310040,0.689960,0.449359
Alpine,0.625000,0.375000,1.666667
Amador,0.595319,0.404681,1.471085
Butte,0.611060,0.388940,1.571089
Calaveras,0.644456,0.355544,1.812592
Colusa,0.643389,0.356611,1.804173
Contra Costa,0.310039,0.689961,0.449358
Del Norte,0.795895,0.204105,3.899441
El Dorado,0.439729,0.560271,0.784850


In [ ]:
# Sort by highest public insurance proportion:
high_public_insurance_counties = public_vs_private.sort_values(by = 'Proportion_Public', ascending = False)
high_public_insurance_counties

,Proportion_Public,Proportion_Private,Public_to_Private_Ratio
County,,,
Del Norte,0.795895,0.204105,3.899441
Modoc,0.742222,0.257778,2.879310
Siskiyou,0.719235,0.280765,2.561702
Trinity,0.710995,0.289005,2.460145
Lake,0.707002,0.292998,2.412990
Shasta,0.681251,0.318749,2.137269
Mendocino,0.669799,0.330201,2.028455
Lassen,0.669476,0.330524,2.025496
Tuolumne,0.658739,0.341261,1.930308


In [ ]:
# Sort by highest private insurance proportion:
high_private_insurance_counties = public_vs_private.sort_values(by = 'Proportion_Private', ascending = False)
high_private_insurance_counties

,Proportion_Public,Proportion_Private,Public_to_Private_Ratio
County,,,
Placer,0.305222,0.694778,0.439309
Contra Costa,0.310039,0.689961,0.449358
Alameda,0.310040,0.689960,0.449359
Sacramento,0.328862,0.671138,0.490007
San Mateo,0.353171,0.646829,0.546003
Sonoma,0.356177,0.643823,0.553221
Yolo,0.360609,0.639391,0.563989
Los Angeles,0.364932,0.635068,0.574634
San Francisco,0.370886,0.629114,0.589536
